# Decomposing unitary matrix into quantum gates

This tool is useful when you have $2^n \times 2^n$ matrix representing a untary operator acting on register of $n$ bits and want to implement this operator in Q#.

This tool decomposes matrix into sequence of two-level unitary matrices using technique proposed in [1] and then implements each two-le


## Example

Consider following matrix:

$$\frac{1}{\sqrt{3}}
\begin{bmatrix}
    1  & 1 & 1 & 0 \\
    1  & e^{\frac{2\pi i}{3}} & e^{\frac{4 \pi i}{3}} & 0 \\
    1  & e^{\frac{4\pi i}{3}} & e^{\frac{2 \pi i}{3}} & 0 \\
    0 & 0 & 0 & \sqrt{3} 
\end{bmatrix}$$

This is $3\times 3$ [DFT matrix](https://en.wikipedia.org/wiki/DFT_matrix), padded to have shape $4 \times 4$. Implementing such matrix was one way to solve problem B2 in [Microsoft Q# Coding Contest - Winter 2019](https://codeforces.com/blog/entry/65579).
[Here](https://assets.codeforces.com/rounds/1116/contest-editorial.pdf) you can find another approach to implementing this matrix, but let's see how we can implement it using our tool and Q#.

First, let's construct this matrix:

In [8]:
import numpy as np
w = np.exp((2j / 3) * np.pi)
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, np.sqrt(3)]]) / np.sqrt(3)
print(A)

[[ 0.57735027+0.j   0.57735027+0.j   0.57735027+0.j   0.        +0.j ]
 [ 0.57735027+0.j  -0.28867513+0.5j -0.28867513-0.5j  0.        +0.j ]
 [ 0.57735027+0.j  -0.28867513-0.5j -0.28867513+0.5j  0.        +0.j ]
 [ 0.        +0.j   0.        +0.j   0.        +0.j   1.        +0.j ]]


Now, let's use quantum_decomp library to construct Q# code.

In [10]:
import quantum_decomp as qd
print(qd.matrix_to_qsharp(A))

operation ApplyUnitaryMatrix (qs : Qubit[]) : Unit {
body (...) {
    Controlled Ry([qs[1]], (-3.141592653589793, qs[0]));
    Controlled R1([qs[1]], (3.141592653589793, qs[0]));
    Controlled Ry([qs[0]], (-1.570796326794897, qs[1]));
    Controlled R1([qs[0]], (3.141592653589793, qs[1]));
    X(qs[1]);
    Controlled Ry([qs[1]], (-1.910633236249018, qs[0]));
    Controlled R1([qs[1]], (3.141592653589793, qs[0]));
    X(qs[1]);
    Controlled Rz([qs[0]], (1.570796326794896, qs[1]));
    Controlled Ry([qs[0]], (-1.570796326794897, qs[1]));
    Controlled Rz([qs[0]], (-1.570796326794896, qs[1]));
    Controlled R1([qs[0]], (3.141592653589793, qs[1]));
    Controlled Rz([qs[1]], (1.570796326794897, qs[0]));
    Controlled Ry([qs[1]], (-3.141592653589793, qs[0]));
    Controlled Rz([qs[1]], (-1.570796326794897, qs[0]));
    Controlled R1([qs[1]], (-1.570796326794897, qs[0]));
  }
}



As you can see from code in qsharp/ directory of this repository, this code indeed implements given unitary matrix. 

Also you can get the same sequence of operations as sequence of gates, where each gate is instance of GateFC or GateSingle, which are internal classes implementing fully controlled gate or gate acting on single qubit.

In [18]:
gates = qd.matrix_to_gates(A)
print('\n'.join(map(str, gates)))

Ry(3.141592653589793) on 0 FC qc=2 fm=0
R1(3.141592653589793) on 0 FC qc=2 fm=0
Ry(1.5707963267948966) on 1 FC qc=2 fm=0
R1(3.141592653589793) on 1 FC qc=2 fm=0
X on 1
Ry(1.9106332362490184) on 0 FC qc=2 fm=0
R1(3.141592653589793) on 0 FC qc=2 fm=0
X on 1
Rz(-1.5707963267948963) on 1 FC qc=2 fm=0
Ry(1.5707963267948966) on 1 FC qc=2 fm=0
Rz(1.5707963267948963) on 1 FC qc=2 fm=0
R1(3.141592653589793) on 1 FC qc=2 fm=0
Rz(-1.5707963267948966) on 0 FC qc=2 fm=0
Ry(3.141592653589793) on 0 FC qc=2 fm=0
Rz(1.5707963267948966) on 0 FC qc=2 fm=0
R1(-1.5707963267948968) on 0 FC qc=2 fm=0


## Output size

Number of Q# commands this tool produces is proportional to number of elements in matrix, which is $O(4^n)$, where $n$ is number of qubits in a register. As it grows very fast, unfortunately this tool is useful only for small $n$.

Let's see how many Q# operations it will generate for t